In [ ]:
# pip3 install ibkr_api
# Ensure ib_insync is installed
# %pip install ib_insync
import ib_insync
from ib_insync import *
import nest_asyncio
import random
from datetime import datetime, timedelta
import pandas as pd

nest_asyncio.apply()

print('version: ' + ib_insync.__version__)

ib: IB = ib_insync.IB()
random_client_id = random.randint(1000, 9999)

try:
    ib.connect('127.0.0.1', 7496, clientId=random_client_id)

    spx = Index('SPX', 'CBOE', 'USD')
    ib.qualifyContracts(spx)
    chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)
    df = util.df(chains)
    if not df.empty:
        print('\nDataframe key:')
        print(df.keys())

        print("\nAvailable Strikes:")
        print(df['strikes'].iloc[0])
        print("\nAvailable expirations:")
        print(df['expirations'].iloc[0]) 
        print('\nAvailable multiplier:')
        print(df['multiplier'])
        print('\nAvailable trading class:')
        print(df['tradingClass'])
    else:
        print("No available chains found.")

    print('\n\n')
    # [Contract(secType='OPT', conId=698122892, symbol='SPX', lastTradeDateOrContractMonth='20250515', strike=5550.0, right='P', multiplier='100', exchange='CBOE', currency='USD', localSymbol='SPX   250516P05550000', tradingClass='SPX'), 
    #  Contract(secType='OPT', conId=772861732, symbol='SPX', lastTradeDateOrContractMonth='20250515', strike=5550.0, right='P', multiplier='100', exchange='CBOE', currency='USD', localSymbol='SPXW  250515P05550000', tradingClass='SPXW')]
    spx_option = Contract(
        secType='OPT', 
        symbol='SPX', 
        lastTradeDateOrContractMonth='20250515', 
        strike=5550.0, 
        right='P', 
        multiplier='100', 
        exchange='CBOE', 
        currency='USD', 
        tradingClass='SPX'
    )
    contracts = ib.qualifyContracts(spx_option)
    if not contracts:
        raise ValueError("Contract not found")

    ticker = ib.reqMktData(
        contracts[0],
        genericTickList='106,100,101,104,106,233,375,377,381,384,387,388', # Generic tick list for options to get Greeks and other data
        # snapshot=False, # snapshot=False to stream data, snapshot=True will return a single snapshot
        snapshot=True, # snapshot=True to get a single snapshot of market data
    )
    for i in range(20):
        ib.sleep(1)
        print(f"Bid: {ticker.bid}, Ask: {ticker.ask}, Last: {ticker.last}")
        if ticker.modelGreeks:
            break

    greeks = {
        'Delta': ticker.modelGreeks.delta if ticker.modelGreeks else None,
        'Gamma': ticker.modelGreeks.gamma if ticker.modelGreeks else None,
        'Theta': ticker.modelGreeks.theta if ticker.modelGreeks else None,
        'Vega': ticker.modelGreeks.vega if ticker.modelGreeks else None,
        'IV': ticker.modelGreeks.impliedVol if ticker.modelGreeks else None
    }
    print('Ticker:', ticker)
    print('Greeks:')
    for key, value in greeks.items():
        print(f'{key}: {value}')

    print('\n\n')
    # APPLE stock
    contract = Stock('AAPL', 'SMART', 'USD')
    # Request market data
    ib.qualifyContracts(contract)
    ticker = ib.reqMktData(contract, '', snapshot=True)

    print('Ticker:', ticker)
    # Request historical data
    end_date = datetime.now()
    start_date = end_date - timedelta(days=30)
    historical_data = ib.reqHistoricalData(
        contract,
        endDateTime=end_date,
        durationStr='30 D',
        barSizeSetting='1 day',
        whatToShow='MIDPOINT',
        useRTH=True,
        formatDate=1
    )
    # Convert to DataFrame
    historical_df = pd.DataFrame(historical_data)
    historical_df['date'] = pd.to_datetime(historical_df['date'])
    historical_df.set_index('date', inplace=True)
    historical_df.sort_index(inplace=True)
    print('Historical Data:')
    print(historical_df)

finally:
    # Disconnect
    ib.disconnect()



version: 0.9.86


positions request timed out



Dataframe key:
Index(['exchange', 'underlyingConId', 'tradingClass', 'multiplier',
       'expirations', 'strikes'],
      dtype='object')

Available Strikes:
[200.0, 400.0, 500.0, 600.0, 800.0, 1000.0, 1200.0, 1400.0, 1600.0, 1800.0, 1900.0, 2000.0, 2100.0, 2200.0, 2300.0, 2400.0, 2500.0, 2600.0, 2700.0, 2800.0, 2900.0, 3000.0, 3050.0, 3100.0, 3150.0, 3200.0, 3250.0, 3300.0, 3350.0, 3400.0, 3450.0, 3500.0, 3525.0, 3550.0, 3575.0, 3600.0, 3625.0, 3650.0, 3675.0, 3700.0, 3725.0, 3750.0, 3775.0, 3800.0, 3825.0, 3850.0, 3875.0, 3900.0, 3925.0, 3950.0, 3975.0, 4000.0, 4025.0, 4050.0, 4075.0, 4080.0, 4090.0, 4100.0, 4110.0, 4120.0, 4125.0, 4130.0, 4140.0, 4150.0, 4160.0, 4170.0, 4175.0, 4180.0, 4190.0, 4200.0, 4210.0, 4220.0, 4225.0, 4230.0, 4240.0, 4250.0, 4260.0, 4270.0, 4275.0, 4280.0, 4290.0, 4300.0, 4310.0, 4320.0, 4325.0, 4330.0, 4340.0, 4350.0, 4360.0, 4370.0, 4375.0, 4380.0, 4390.0, 4400.0, 4410.0, 4420.0, 4425.0, 4430.0, 4440.0, 4450.0, 4460.0, 4470.0, 4475.0, 4480.0, 4490.0, 4500

TypeError: IB.reqMktData() got multiple values for argument 'snapshot'